# Create Merged Dataset

In this workbook, you will read in the `trip` and `fare` files. You are welcome to use DataFrame and/or SparkSQL API as you desire as long as it produces the expected results.

Instructions:

1. Join both datasets such that you get a merged dataset with 21 unique fields. You need to determine how to join the dataset.
2. Once you create the merged dataset, you need to convert fields to the following types, since all fields were read is as string:
    * pickup_datetime and dropoff_datetime must be TIMESTAMP
    * passenger_count and rate_code must be INT
    * all other numeric fields must be FLOAT
    * the remaining fields stay as STRING
3. Save your merged and converted dataset to your own S3 bucket in parquet format.

You are welcome to add as many cells as you need below up until the next section. **You must include comments in your code.**

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from functools import reduce
from pyspark.sql import functions as f
spark = SparkSession.builder.appName("lab-ml").getOrCreate()

In [3]:
# Loading Parquet files into dataframe.
# might need to change csv to parquet when reading in the full data set.  
trip_df = spark.read\
  .format('csv')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .parquet('s3://bigdatateaching/nyctaxi-2013/parquet/trip')
fare_df = spark.read.format("csv").option('header', 'true').option('inferSchema', 'true')\
    .parquet('s3://bigdatateaching/nyctaxi-2013/parquet/fare')

In [4]:
# Creating unqiue id for each observation.  
trip_df = trip_df.withColumn('id', monotonically_increasing_id())
fare_df = fare_df.withColumn('id', monotonically_increasing_id())

In [5]:
# Combine the two dataframes on id column.
combined = trip_df.join(fare_df,on='id', how='full_outer').select(
trip_df.medallion,
trip_df.hack_license,
trip_df.vendor_id,
trip_df.rate_code.cast('int'),
trip_df.store_and_fwd_flag,
trip_df.pickup_datetime.cast('timestamp'),
trip_df.dropoff_datetime.cast('timestamp'),
trip_df.passenger_count.cast('int'),
trip_df.trip_time_in_secs.cast('float'),
trip_df.trip_distance.cast('float'),
trip_df.pickup_longitude.cast('float'),
trip_df.pickup_latitude.cast('float'),
trip_df.dropoff_longitude.cast('float'),
trip_df.dropoff_latitude.cast('float'),
fare_df.payment_type,
fare_df.fare_amount.cast('float'),
fare_df.surcharge.cast('float'),
fare_df.mta_tax.cast('float'),
fare_df.tip_amount.cast('float'),
fare_df.tolls_amount.cast('float'),
fare_df.total_amount.cast('float'))

In [6]:
# Creating tip label.  1 = passenger tipped; 0 = no tip.
combined = combined.withColumn('tip', f.when(f.col('tip_amount') > 0, 1).otherwise(0))

## In the following cells, please provide the requested code and output. Do not change the order and/or structure of the cells.

In the next cell, provide the code that saves your merged dataset to your S3 bucket.

In [7]:
# Save data to gu-502-course s3 bucket.
combined.write.parquet("s3://gu-502-course/trip_fare_combined", mode="overwrite")

In the next cell, print the schema of your merged dataset.

In [8]:
combined.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: float (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- tip: integer (nullable = false)



In the next cell, print the first 10 records of your merged dataset.

In [9]:
combined.show(10)

+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+---+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|tip|
+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+---+
|75C7F961C2AB3EFE9...|A9DD60890

In the next cell, print the row count of your merged dataset.

In [10]:
combined.count()

186289125